In [59]:
#export
import sys
if __name__ == '__main__': sys.path.append('..')
import download_youtube_subtitle.common as common
from pprint import pprint
def pj(*args, **kargs):
    if common.IN_JUPYTER:
        pprint(*args, **kargs)
        
from functools import partial
import sys

perr = partial(print, "ERR: ")

In [60]:
#export
import requests
import socket
socket.setdefaulttimeout(5.)

In [61]:
#export
from pytube import YouTube
def get_tracks_title(videoID):
    yt = YouTube(f"http://youtube.com/watch?v={videoID}")
    yt.streams
    return yt.caption_tracks, yt.title, yt

In [62]:
_captions,_title, yt =get_tracks_title("2lAe1cqCOXo")
print(_title, _captions[:2])

YouTube Rewind 2019: For the Record | #YouTubeRewind [<Caption lang="Arabic" code="ar">, <Caption lang="Chinese (Hong Kong)" code="zh-HK">]


In [63]:
# _captions[0].xml_captions

In [64]:
#export
# add tlang=zh-Hans to baseUrl
# will get translation

# getting track info

class CaptionNotAvailableException(Exception):
    pass 
import re
import json
import urllib

In [83]:
#export
# dealing with transcript
import math
from functools import partial
import sys
from xml.dom.minidom import parseString
import html
def parseTranscript(transcript, remove_font_tag=True):
    dom = parseString(transcript)
    texts = dom.getElementsByTagName('p')
    
    _eachTxt = partial(eachTxt, remove_font_tag=remove_font_tag)
    texts = list(map( _eachTxt, texts,))
    return texts 

def each_sent(o, file=sys.stdout):
    if len(o['text'].strip()) == 0 and len(o.get('translate_text', "").strip()) == 0:
        return
    start = o['start']
    start = float(start)
    start /= 1000
    minute = math.floor(start/60)
    second = math.floor(start%60)
    p = partial(print, file=file)
    p(f"---------{minute:02d}:{second:02d}----------")
    p(o['text'])
    translate_text = o.get('translate_text', None)
    if translate_text:
        p(translate_text)
   

In [66]:
#export
# dealing with valid filename
# https://github.com/django/django/blob/master/django/utils/text.py
import re
def get_valid_filename(s):
    """
    Return the given string converted to a string that can be used for a clean
    filename. Remove leading and trailing spaces; convert other spaces to
    underscores; and remove anything that is not an alphanumeric, dash,
    underscore, or dot.
    >>> get_valid_filename("john's portrait in 2004.jpg")
    'johns_portrait_in_2004.jpg'
    """
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [67]:
#export
import copy
def merge_subtitle(subtitle, subtitle_cn):
    """
    merge subtitle_cn(traslation) to subtitle(origin).
    cc and translated cc aren't always align, 
    I found at least in cn and ja, translated cc are less than cc
    see  videoID='HSz7Q4YnQ_M'
    cc and translated cc aren't always equal in time see wgNiGj1nGYE for translation ja
    """
    subtitle = copy.deepcopy(subtitle) # original transcript
    subtitle_cn = copy.deepcopy(subtitle_cn) # translation script
    
    TRANSLATE_TEXT="translate_text"
    TEXT="text"
    START="start"
    
    # NOTE not sure how to merge them properly
    
    # indexer for subtitle 
    sub_p = 0
    sub_p_cn = 0
    
    while sub_p < len(subtitle) and sub_p_cn < len(subtitle_cn):
        
        sub = subtitle[sub_p]
        sub_cn = subtitle_cn[sub_p_cn]
        
        if TRANSLATE_TEXT not in sub: sub[TRANSLATE_TEXT] = ""
        
        if float(sub[START]) >= float(sub_cn[START]) :
        # sub goes first, being chased by sub_cn
        
            # for separating the sentence
            if len(sub[TRANSLATE_TEXT]) != 0: sub[TRANSLATE_TEXT] += "\n"
            
            sub[TRANSLATE_TEXT] +=  sub_cn[TEXT]
            
            sub_p_cn += 1
            
        else :
            sub_p += 1
            
    # add empty field for subitle
    while sub_p < len(subtitle):
        assert sub_p_cn == len(subtitle_cn)
        
        sub = subtitle[sub_p]
        if TRANSLATE_TEXT not in sub: sub[TRANSLATE_TEXT] = ""
        sub_p += 1
        
    # add the rest of subtitle_cn to the last one of subtitle
    while sub_p_cn < len(subtitle_cn):
        assert sub_p == len(subtitle)
        
        sub = subtitle[-1]
        
        if TRANSLATE_TEXT not in sub: sub[TRANSLATE_TEXT] = ""
            
        if len(sub[TRANSLATE_TEXT]) != 0: sub[TRANSLATE_TEXT] += "\n"
        
        sub[TRANSLATE_TEXT] += sub_cn[TEXT]
        sub_p_cn += 1
        
    assert sub_p == len(subtitle)
    assert sub_p_cn == len(subtitle_cn)
        
    return subtitle

In [68]:
#export
# dealing with xml.dom
import re
import xml
def eachTxt(txt, remove_font_tag):
    start = txt.attributes.get("t").value
    dur = '0' if txt.attributes.get("d") is None \
        else txt.attributes.get("d").value 
    if txt.firstChild is None:
        txt = ""
    elif isinstance(txt.firstChild, xml.dom.minidom.Text):
        txt = txt.firstChild.data
    elif txt.firstChild.tagName == 's':
        ret_txt = ""
        for sTxt in txt.childNodes:
            if sTxt.firstChild: ret_txt += sTxt.firstChild.data
        txt = ret_txt
    
    
    txt = html.unescape((txt))
    if remove_font_tag:
        txt =   re.sub(r'</?font[^>]*>','', txt)
        
    return {
        "start":start, 
        "dur": dur,
        "text": txt
    }

In [69]:
#test_export
# here to check parsing dom
videoID="SVdQ-2njL6U" # test [Fix issue where translation is not True by marosoft · Pull Request #3 · xsthunder/download-youtube-subtitle](https://github.com/xsthunder/download-youtube-subtitle/pull/3)
captionTracks, title, yt = get_tracks_title(videoID)
# baseUrl = captionTracks[0]['baseUrl']
# transcript = requests.get(baseUrl)
# dom = parseString(transcript.text)
dom = parseString(captionTracks[0].xml_captions)
texts = dom.getElementsByTagName('p')

_eachTxt = partial(eachTxt, remove_font_tag=True)
for i,t in enumerate(texts):
    if i>5:break
    print(t, _eachTxt(t))

<DOM Element: p at 0x7ff0b6a3fa60> {'start': '140', 'dur': '4570', 'text': 'hi this is Louis from Rossville repair'}
<DOM Element: p at 0x7ff0b6a3f550> {'start': '2960', 'dur': '1750', 'text': '\n'}
<DOM Element: p at 0x7ff0b6a3f0d0> {'start': '2970', 'dur': '3869', 'text': 'group in this video in a couple of'}
<DOM Element: p at 0x7ff0b68f1040> {'start': '4700', 'dur': '2139', 'text': '\n'}
<DOM Element: p at 0x7ff0b68f10d0> {'start': '4710', 'dur': '4469', 'text': "minutes you're going to watch me as I"}
<DOM Element: p at 0x7ff0b68f15e0> {'start': '6829', 'dur': '2350', 'text': '\n'}


In [70]:
_eachTxt(texts[1])['text'].strip()

''

In [71]:
o = _eachTxt(texts[1])
len(o['text'].strip()) == 0 and len(o.get('translate_text', "").strip())

0

In [72]:
print(dom.toprettyxml()[:1000])

<?xml version="1.0" ?>
<timedtext format="3">
	

	<head>
		

		<ws id="0"/>
		

		<ws id="1" mh="2" ju="0" sd="3"/>
		

		<wp id="0"/>
		

		<wp id="1" ap="6" ah="20" av="100" rc="2" cc="40"/>
		

	</head>
	

	<body>
		

		<w t="0" id="1" wp="1" ws="1"/>
		

		<p t="140" d="4570" w="1">
			<s ac="252">hi</s>
			<s t="1000" ac="201"> this</s>
			<s t="1480" ac="201"> is</s>
			<s t="1540" ac="146"> Louis</s>
			<s t="1989" ac="245"> from</s>
			<s t="2200" ac="217"> Rossville</s>
			<s t="2410" ac="227"> repair</s>
		</p>
		

		<p t="2960" d="1750" w="1" a="1">
</p>
		

		<p t="2970" d="3869" w="1">
			<s ac="239">group</s>
			<s t="179" ac="187"> in</s>
			<s t="360" ac="252"> this</s>
			<s t="810" ac="208"> video</s>
			<s t="1110" ac="235"> in</s>
			<s t="1349" ac="252"> a</s>
			<s t="1440" ac="157"> couple</s>
			<s t="1710" ac="252"> of</s>
		</p>
		

		<p t="4700" d="2139" w="1" a="1">
</p>
		

		<p t="4710" d="4469" w="1">
			<s ac="236">minutes</s>
			<s t="270" ac="252"> you

In [79]:
for o in list(yt.captions.keys()):
    print(o.code)

zh-Hans
zh-Hant
en-US
fr
de
id
pt
ru
es
th
vi


In [80]:
yt.captions['vi']

<Caption lang="Vietnamese" code="vi">

In [90]:
#test_export
#here to break up the procss
# videoID="Zd14s2WW-Tc"
videoID="wgNiGj1nGYE"
captionTracks, title, yt = get_tracks_title(videoID)
subtitle = parseTranscript(yt.captions['en-US'].xml_captions)
subtitle_cn = parseTranscript(yt.captions['zh-Hans'].xml_captions)
merged_subtitle = merge_subtitle(subtitle, subtitle_cn)
merged_subtitle[:4]

[{'start': '0',
  'dur': '5056',
  'text': 'Once upon a time, two twins traveled together throughout the universe.',
  'translate_text': '从前，有一对双胞胎结伴在宇宙中旅行'},
 {'start': '5398',
  'dur': '2398',
  'text': 'But one day, their path was blocked,',
  'translate_text': '但有一天，他们前路遇阻'},
 {'start': '7988',
  'dur': '2801',
  'text': 'and they were separated by an unknown god.',
  'translate_text': '被一个未知的神明生生分离'},
 {'start': '11745', 'dur': '804', 'text': 'Aether! ', 'translate_text': '空！'}]

In [119]:
#export
def format_caption(caption):
    name = caption.name
    pattern = 'code='
    code = caption.__repr__()
    code = code[code.find(pattern):-1]
    ret = f"{code:8s} name=\"{name}\""
    if 'a.' in code:
        ret += ' automatically generated by youtube'
    return ret


<Caption lang="Chinese (Traditional)" code="zh-Hant">


'code="zh-Hant" name="Chinese (Traditional)"'

In [113]:
print(captionTracks[1])
format_caption(captionTracks[1])
pattern = 'code='
code = caption.__repr__()
code = code[code.find(pattern):-1]
code

'code="zh-Hans'

In [120]:
#test_export
#here test output
output_file = get_valid_filename(f'{title}.txt')
with open(output_file , 'w', encoding='UTF-8') as f:
    print("save to ", output_file)
    for sent in subtitle:
        each_sent(sent, file=f)

save to  Version_1.5_Special_ProgramGenshin_Impact.txt


In [121]:
#export
def parseVideoID(videoID):
    if 'youtu' in videoID:
        videoID = re.search('v=([^&]+)', videoID).group(1)
        
    video_link = f'https://www.youtube.com/watch?v={videoID}'
    return videoID, video_link

In [122]:
#test_export
videoID = 'https://www.youtube.com/watch?v=wgNiGj1nGYE'
videoID, video_link= parseVideoID(videoID)
videoID = 'wgNiGj1nGYE'
assert (videoID, video_link) == parseVideoID(videoID)

In [202]:
#export
import fire
import sys
from functools import partial
import json
import re
from typing import Union, Optional
def main(
    videoID:str, 
    translation:str=None, 
    caption_num:int=0, 
    caption_num_second:int=None,
    output_file:str=None, 
    save_to_file:bool=True, 
    to_json:bool=False, 
    remove_font_tag:bool=True, 
    ):
    """
    download youtube closed caption(subtitles) by videoID
    
    Examples:
    dl-youtube-cc -h # to see this helpful infomation
    dl-youtube-cc wgNiGj1nGYE --translation 'ru' # use russian translation, see ./lang_code for full list
    dl-youtube-cc wgNiGj1nGYE --caption_num=1 --translation 'ru' # choose the caption num for original transcript and use russian translation,
    dl-youtube-cc wgNiGj1nGYE --caption_num=1 --caption_num_second=2 # manually choose the original and translation transcript from available caption list
    dl-youtube-cc wgNiGj1nGYE --translation False # without translation
    dl-youtube-cc wgNiGj1nGYE --save_to_file=False # print stuff in console
    dl-youtube-cc wgNiGj1nGYE --output_file='test.txt' # print stuff in named file
    dl-youtube-cc wgNiGj1nGYE --to_json=True # print stuff in json
    
    Argument:
    videoID : the video link or the id of youtube video, the string after 'v=' in a youtube video link
    translation : which will be displayed as original transcript, default to None
    caption_num : choose the caption which will be displayed as original transcript
    caption_num_second : will surpass translation option, choose the caption which will be displayed as translation transcript
    output_file : default to video title
    save_to_file : pass False to print in console
    to_json: pass True to export caption to json
    remove_font_tag: remove font tag
    """    
    videoID, video_link = parseVideoID(videoID)
    
    captionTracks, title, yt = get_tracks_title(videoID)
    
    info = partial(print, "INFO: ")
    
    info("available caption(s):")
    for i, caption in enumerate(captionTracks):
        mark = '⭕'
        if caption_num == i:mark = '✔ as original'
        if translation and translation in format_caption(caption): mark = '✔ as translation' 
        if caption_num_second == i: mark = '✔ as translation' 
        notice = f"#{i:<2} {mark}"
        info(notice, format_caption(caption))
    info("given by --caption_num default to 0 as original")
    if caption_num_second is None: info("you may use --caption_num_second intead of --translation to spefify translation transript")
    
    _parseTranscript = partial(parseTranscript, remove_font_tag=remove_font_tag)
    subtitle = _parseTranscript(captionTracks[caption_num].xml_captions, )
    sent_subtitle = subtitle
    
    output_json = { "original": subtitle }
    
    if translation or caption_num_second:
        if translation:
            translation_caption=yt.captions[translation]
            # baseUrl = caption['baseUrl'] + '&tlang=' + translation
        if caption_num_second: 
            translation_caption=captionTracks[caption_num_second]
            # translation = False
            
            # baseUrl = get_caption_url(caption_num_second)
        # transcript = requests.get(baseUrl)
        subtitle_cn = _parseTranscript(translation_caption.xml_captions)
        merged_subtitle = merge_subtitle(subtitle, subtitle_cn)
        sent_subtitle = merged_subtitle
        output_json = {
                 "original": subtitle,
                 "translation":subtitle_cn,
                # note that it's not guaranteed to be aligned.
                "merged": merged_subtitle,
            }
        
    ######################## save to file
            
    f = sys.stdout
    if save_to_file :
        if output_file is None: 
            if to_json:
                output_file = get_valid_filename(f'{title}.json')
            else:
                output_file = get_valid_filename(f'{title}.txt')
        f = open(output_file , 'w', encoding='UTF-8')
        info("Save to ", output_file )
    
    if to_json:
        json.dump(
            output_json
             , f, indent=4, ensure_ascii=False)
        return 
    
    pfile = partial(print, file=f)
    pfile("video_link\t", video_link)
    pfile("original\t", format_caption(captionTracks[caption_num]))
    if translation: pfile("translation\t",translation)
    if caption_num_second: pfile("translation\t", format_caption(captionTracks[caption_num_second]))
    for sent in sent_subtitle:
        each_sent(sent, file=f)
        pfile()

In [174]:
#test_export
# see [Fix issue where translation is not True by marosoft · Pull Request #3 · xsthunder/download-youtube-subtitle](https://github.com/xsthunder/download-youtube-subtitle/pull/3)
main("SVdQ-2njL6U", translation=False, output_file=f'a.txt')

INFO:  available caption(s):
INFO:  #0  ✔ as original code="a.en" name="English (auto-generated)" automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  a.txt


In [175]:
#export
from functools import partial
def set_fire(fn):
    if common.IN_TRAVIS or common.IN_JUPYTER:
        return 
    fire.Fire(fn)
if __name__ == '__main__':
    if common.IN_TRAVIS or common.IN_JUPYTER:
        pass
    else :
        set_fire(main)
fire_main = partial(set_fire, main)

In [176]:
#test_export
main("wgNiGj1nGYE", translation='fr', output_file="fr.txt")

INFO:  available caption(s):
INFO:  #0  ✔ as original code="zh-Hans" name="Chinese (Simplified)"
INFO:  #1  ⭕ code="zh-Hant" name="Chinese (Traditional)"
INFO:  #2  ⭕ code="en-US" name="English (United States)"
INFO:  #3  ⭕ code="fr" name="French"
INFO:  #4  ⭕ code="de" name="German"
INFO:  #5  ⭕ code="id" name="Indonesian"
INFO:  #6  ⭕ code="pt" name="Portuguese"
INFO:  #7  ⭕ code="ru" name="Russian"
INFO:  #8  ⭕ code="es" name="Spanish"
INFO:  #9  ⭕ code="th" name="Thai"
INFO:  #10 ⭕ code="vi" name="Vietnamese"
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  fr.txt


In [177]:
print(f"{2:<2}", 1)

2  1


In [178]:
#test_export
main("wgNiGj1nGYE", caption_num=0, caption_num_second=7, output_file="zh_ru.txt")

INFO:  available caption(s):
INFO:  #0  ✔ as original code="zh-Hans" name="Chinese (Simplified)"
INFO:  #1  ⭕ code="zh-Hant" name="Chinese (Traditional)"
INFO:  #2  ⭕ code="en-US" name="English (United States)"
INFO:  #3  ⭕ code="fr" name="French"
INFO:  #4  ⭕ code="de" name="German"
INFO:  #5  ⭕ code="id" name="Indonesian"
INFO:  #6  ⭕ code="pt" name="Portuguese"
INFO:  #7  ✔ as translation code="ru" name="Russian"
INFO:  #8  ⭕ code="es" name="Spanish"
INFO:  #9  ⭕ code="th" name="Thai"
INFO:  #10 ⭕ code="vi" name="Vietnamese"
INFO:  given by --caption_num default to 0 as original
INFO:  Save to  zh_ru.txt


In [179]:
#test_export
main("wgNiGj1nGYE", caption_num=0, caption_num_second=3, output_file="0,3-zh,fr.txt")

INFO:  available caption(s):
INFO:  #0  ✔ as original code="zh-Hans" name="Chinese (Simplified)"
INFO:  #1  ⭕ code="zh-Hant" name="Chinese (Traditional)"
INFO:  #2  ⭕ code="en-US" name="English (United States)"
INFO:  #3  ✔ as translation code="fr" name="French"
INFO:  #4  ⭕ code="de" name="German"
INFO:  #5  ⭕ code="id" name="Indonesian"
INFO:  #6  ⭕ code="pt" name="Portuguese"
INFO:  #7  ⭕ code="ru" name="Russian"
INFO:  #8  ⭕ code="es" name="Spanish"
INFO:  #9  ⭕ code="th" name="Thai"
INFO:  #10 ⭕ code="vi" name="Vietnamese"
INFO:  given by --caption_num default to 0 as original
INFO:  Save to  0,3-zh,fr.txt


In [203]:
#test_export
main("wgNiGj1nGYE", caption_num=0, translation="en-US", to_json=True, output_file="0,en.txt")

INFO:  available caption(s):
INFO:  #0  ✔ as original code="zh-Hans" name="Chinese (Simplified)"
INFO:  #1  ⭕ code="zh-Hant" name="Chinese (Traditional)"
INFO:  #2  ✔ as translation code="en-US" name="English (United States)"
INFO:  #3  ⭕ code="fr" name="French"
INFO:  #4  ⭕ code="de" name="German"
INFO:  #5  ⭕ code="id" name="Indonesian"
INFO:  #6  ⭕ code="pt" name="Portuguese"
INFO:  #7  ⭕ code="ru" name="Russian"
INFO:  #8  ⭕ code="es" name="Spanish"
INFO:  #9  ⭕ code="th" name="Thai"
INFO:  #10 ⭕ code="vi" name="Vietnamese"
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  0,en.txt


In [181]:
%pdb on 

Automatic pdb calling has been turned ON


In [182]:
#test_export
#fix eachTxt, allow txt.firstChild = None
main('https://www.youtube.com/watch?v=Zd14s2WW-Tc', caption_num=1)

INFO:  available caption(s):
INFO:  #0  ⭕ code="a.en" name="English (auto-generated)" automatically generated by youtube
INFO:  #1  ✔ as original code="en.j3PyPqV-e1s" name="English - CC (English)"
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Chinese_Politics__Authoritarian_Regimes__Professor_Rory_Truex__Talks_at_Google.txt


In [198]:
#test_export
import sure
from pytube import exceptions
main.when.called_with('HSz7Q4YnQ_M').throw(exceptions.AgeRestrictedError) # fix when cc's length and translated cc's don't match
main.when.called_with('https://www.youtube.com/watch?v=EozTm6ZVf1U', caption_num=1).throw(exceptions.AgeRestrictedError)

ERR:  can't not retrive caption,   for detail
ERR:  can't not retrive caption,   for detail


True

In [185]:
#test_export
main('MhCEdIqFCck') # fix title with quotes "

INFO:  available caption(s):
INFO:  #0  ✔ as original code="en" name="English"
INFO:  #1  ⭕ code="a.en" name="English (auto-generated)" automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Dont_Lose_Your_Accent__Learning_Accents_-_TREVOR_NOAH_Pay_Back_The_Funny.txt


In [187]:
#test_export
def read_all_content(f):
    with open(f, 'r', encoding='UTF-8') as f:
        ret = ''.join(f.readlines())
    return ret

In [188]:
#test_export
f = 'no_font_tag.txt'
main('tktbVrTFUkc', output_file=f) # remove font tag
assert '<font' not in read_all_content(f)
assert '</font>' not in read_all_content(f)
del f

INFO:  available caption(s):
INFO:  #0  ✔ as original code="a.en" name="English (auto-generated)" automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  no_font_tag.txt


In [189]:
#test_export
f = 'save_font_tag.txt'
main('tktbVrTFUkc', remove_font_tag=False, output_file=f) # remove font tag
del f

INFO:  available caption(s):
INFO:  #0  ✔ as original code="a.en" name="English (auto-generated)" automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  save_font_tag.txt


In [205]:
common.save_and_export_notebook('main')

<IPython.core.display.Javascript object>

Converted main.ipynb to ../download_youtube_subtitle/main.py
Converted main.ipynb to ../tests/test_main.py


<IPython.core.display.Javascript object>